In [48]:
import pandas as pd
from Bio import SeqIO
import pickle

In [49]:
DEBUG = False

# OrthoGroups to MSA 
+ load table with the proteins - orthogrous distribution 
+ filter the core orthogroups 
+ load proteins .pkl database
+ get the proteins and the genome positions corresponding to the core OGs

In [50]:
core_ogs = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_rso_test/Species_Tree/Orthogroups_for_concatenated_alignment.txt'
ogs2prot = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_rso_test/Orthogroups/Orthogroups.tsv'
prot_db = pd.read_pickle("../outputs/GENOME_PROTEIN_DB_rso_test.pkl")
path_p_aln = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_rso_test/MultipleSequenceAlignments/'

msa_out_fasta = "/Users/devseeva/Desktop/work/sm_workflow/snakefiles/rso_test_concat.fasta"
msa_out_phylip = "/Users/devseeva/Desktop/work/sm_workflow/snakefiles/rso_test_concat.phylip"
msa_out_xmfa = "/Users/devseeva/Desktop/work/sm_workflow/snakefiles/rso_test_concat.xmfa"
aln2acc = "/Users/devseeva/Desktop/work/sm_workflow/snakefiles/outputs/MSA_ACC_DB_rso_test.pkl"

#core_ogs = snakemake.input["core_ogs"]
#ogs2prot = snakemake.input["ogs2prot"]
#prot_db = pd.read_pickle(snakemake.input["prot_db"])
#path_p_aln = snakemake.input["path2aln"]

#msa_out_fasta = snakemake.output["msa_fasta"]
#msa_out_phylip = snakemake.output["msa_phylip"]
#msa_out_xmfa = snakemake.output["msa_xmfa"]

In [51]:
# get core ogs
f = open(core_ogs,"r")
ogs = []
for l in f:
    ogs.append(l.replace('\n',''))
f.close()
##print(ogs)

In [52]:
# load roteins ids from the OGs table 
# keep only core the OGs

tab_prot = pd.read_csv(ogs2prot, sep='\t', index_col=0)  
tab_prot = tab_prot.loc[ogs]
tab_prot

,GCF_001887535.1_ASM188753v1_protein,GCF_013306235.1_ASM1330623v1_protein,GCF_013306335.1_ASM1330633v1_protein,GCF_013306435.1_ASM1330643v1_protein,GCF_013306935.1_ASM1330693v1_protein,GCF_013375735.1_ASM1337573v1_protein
Orthogroup,,,,,,
OG0000225,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1
OG0000226,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1
OG0000227,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1
OG0000228,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1
OG0000229,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1
...,...,...,...,...,...,...
OG0004089,WP_155772886.1,WP_148668905.1,WP_148668915.1,WP_148668915.1,WP_148668915.1,WP_155772886.1
OG0004091,WP_155773184.1,WP_173940865.1,WP_173941733.1,WP_119889930.1,WP_173952290.1,WP_086706436.1
OG0004092,WP_155773186.1,WP_144061950.1,WP_119417730.1,WP_119889926.1,WP_119417730.1,WP_038938529.1


In [53]:
genomes_for_og = list(tab_prot.columns) 
genomes_for_og

['GCF_001887535.1_ASM188753v1_protein',
 'GCF_013306235.1_ASM1330623v1_protein',
 'GCF_013306335.1_ASM1330633v1_protein',
 'GCF_013306435.1_ASM1330643v1_protein',
 'GCF_013306935.1_ASM1330693v1_protein',
 'GCF_013375735.1_ASM1337573v1_protein']

# Get Nucleotide sequences + corresponding positions in genome

In [54]:
prot_db

,GCF_013306935.1_ASM1330693v1_protein,GCF_013306235.1_ASM1330623v1_protein,GCF_013306435.1_ASM1330643v1_protein,GCF_001887535.1_ASM188753v1_protein,GCF_013306335.1_ASM1330633v1_protein,GCF_013375735.1_ASM1337573v1_protein
WP_071507848.1,"{'chr': 'NZ_CP052126.1', 'location': (0, 1, 2,...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'location': (0, 1, 2,...",NaN
WP_013210791.1,"{'chr': 'NZ_CP052126.1', 'location': (1846, 18...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'location': (1846, 18...",NaN
WP_173941386.1,"{'chr': 'NZ_CP052126.1', 'location': (3098, 30...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'location': (3098, 30...",NaN
WP_173941387.1,"{'chr': 'NZ_CP052126.1', 'location': (6305, 63...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'location': (6305, 63...",NaN
WP_086005422.1,"{'chr': 'NZ_CP052127.1', 'location': (666300, ...",NaN,"{'chr': 'NZ_CP052097.1', 'location': (693036, ...","{'chr': 'NZ_CP016555.1', 'location': (1164461,...","{'chr': 'NZ_CP052087.1', 'location': (666300, ...","{'chr': 'NZ_CP056086.1', 'location': (1889533,..."
...,...,...,...,...,...,...
WP_016726112.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'location': (1945036,..."
WP_028861767.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'location': (1973916,..."
WP_023470131.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'location': (1980032,..."
WP_043885778.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'location': (1985998,..."


In [55]:
#prot_db.loc['WP_003261125.1', 'GCF_000215325_1_ASM21532v1_protein']
#"WP_003261125.1" in prot_db.index

# Load single proteins alignments and extract gaps

In [56]:
gaps2tab = pd.DataFrame(columns=tab_prot.columns, index=tab_prot.index)
#gaps2tab

In [57]:
for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    for record in SeqIO.parse(pmsa, "fasta"):  
        #print(record.id)
        
        header = record.id.split('_WP_')
        genome_id = header[0].replace('_1_', '.1_')
        protien_id = 'WP_' + header[1]
        
        # EXACT GAPS MAPPING 
        gaps_bool = []
        for aa in record.seq:
            if aa == '-':
                gaps_bool.append(1) 
                gaps_bool.append(1) 
                gaps_bool.append(1) 
            else:
                gaps_bool.append(0) 
                gaps_bool.append(0) 
                gaps_bool.append(0)
        
        prot_db.loc[protien_id, genome_id]['aln_gaps'] = gaps_bool
        gaps2tab.loc[og][genome_id] = gaps_bool
gaps2tab

,GCF_001887535.1_ASM188753v1_protein,GCF_013306235.1_ASM1330623v1_protein,GCF_013306335.1_ASM1330633v1_protein,GCF_013306435.1_ASM1330643v1_protein,GCF_013306935.1_ASM1330693v1_protein,GCF_013375735.1_ASM1337573v1_protein
Orthogroup,,,,,,
OG0000225,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000226,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000227,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000228,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000229,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
OG0004089,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0004091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0004092,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Genome-based solution
+ for each genome grab the cols of the core OG table
+ concatenate the corresponding nucleotides from the GENOME_PROTEIN_DB

## TODO length of nucleotides is larger than length of translation*3 ?!

In [60]:
g2nucl = {}
g2pos = {}
g2chr = {}
g2strand = {}
g2acc = {}
g2gaps = {}

nucl_fasta = open("core_OG_nucl.fasta", "w")

for g in genomes_for_og:
    print('Genome', g)
    
    #if g == 'GCF_001586135.1_ASM158613v1_protein' or g == "GCF_001587135.1_ASM158713v1_protein":
    #    continue
    
    # get corresponding nucleotide positions 
    g2pos[g] = []
    # get corresponding chr ids 
    g2chr[g] = []
    # get corresponding strands
    g2strand[g] = []
    # get protein accession for each genome position
    g2acc[g.split('.1_')[1][:10]] = []
    
    g2gaps[g] = []
    
    # get nucleotide sequences for all the core og proteins per genome 
    nucl_per_genome = []
    
    for i in prot_db.loc[tab_prot[g].values, g]: 
        nucl_per_genome.append(str(i['nucl']))
        g2pos[g].extend(i['location'])
        #g2chr[g].append((i['chr'], len(i['nucl'])))
        prot_len = len(i['nucl']) ######################## <-----
        for counter in range(len(i['aln_gaps'])):
            if DEBUG:
                g2chr[g].append(i['chr'])
                g2strand[g].append(i['location'].strand)
            g2acc[g.split('.1_')[1][:10]].append(i['acc']) # phylip name format
        g2gaps[g].extend(i['aln_gaps'])
    
    nucl_per_genome = "".join(nucl_per_genome)
    g2nucl[g] = nucl_per_genome
    print('Exported sequence length', len(nucl_per_genome))
    print('Exported protein accessions', len(g2acc[g.split('.1_')[1][:10]]))
    print('Exported alignment gaps', len(g2gaps[g]))
    print('Exported original positions', len(g2pos[g]))
    print('Exported chr labels', len(g2chr[g]))
    print('Exported strands', len(g2strand[g]))
    nucl_fasta.write('>'+g+'\n')
    nucl_fasta.write(nucl_per_genome+'\n')
    print()
    
nucl_fasta.close()

Genome GCF_001887535.1_ASM188753v1_protein
Exported sequence length 3827778
Exported protein accessions 3825198
Exported alignment gaps 3825198
Exported original positions 3827778
Exported chr labels 0
Exported strands 0

Genome GCF_013306235.1_ASM1330623v1_protein
Exported sequence length 3827919
Exported protein accessions 3825198
Exported alignment gaps 3825198
Exported original positions 3827919
Exported chr labels 0
Exported strands 0

Genome GCF_013306335.1_ASM1330633v1_protein
Exported sequence length 3824796
Exported protein accessions 3825198
Exported alignment gaps 3825198
Exported original positions 3824796
Exported chr labels 0
Exported strands 0

Genome GCF_013306435.1_ASM1330643v1_protein
Exported sequence length 3826515
Exported protein accessions 3825198
Exported alignment gaps 3825198
Exported original positions 3826515
Exported chr labels 0
Exported strands 0

Genome GCF_013306935.1_ASM1330693v1_protein
Exported sequence length 3824484
Exported protein accessions 3825

In [61]:
# SAVE PROTEIN ACC FOR MAPPING

with open(aln2acc, 'wb') as handle:
    pickle.dump(g2acc, handle, protocol=pickle.HIGHEST_PROTOCOL)

# replace ambiguous bases with gaps!

In [62]:
for g in g2nucl:
    atcg = {'A', 'C', 'T', 'G'}
    ambig = set(g2nucl[g]).difference(atcg)
    
    print("before", set(g2nucl[g]))
    for b in ambig:
        g2nucl[g] = g2nucl[g].replace(b, '-')
    print("after ",set(g2nucl[g]))

before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}
before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}
before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}
before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}
before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}
before {'T', 'C', 'G', 'A'}
after  {'T', 'C', 'G', 'A'}


In [63]:
#str(prot_db.loc["WP_097388881.1","GCF_002501565.1_ASM250156v1_protein"]['nucl'])
#for v in prot_db['GCF_001587135.1_ASM158713v1_protein'].values:
#    print(v)

# Export MSA

In [64]:
msa_fasta = open(msa_out_fasta, "w")
for g in g2nucl:
    print(g, len(g2nucl[g]))
    msa_fasta.write('>'+g.split('_')[2][:10]+'\n')
    
    i = 0
    for gap in g2gaps[g]:
        if gap:
            msa_fasta.write('-')
        else:
            msa_fasta.write(g2nucl[g][i])
            i = i + 1
    msa_fasta.write('\n')
msa_fasta.close()

GCF_001887535.1_ASM188753v1_protein 3827778
GCF_013306235.1_ASM1330623v1_protein 3827919
GCF_013306335.1_ASM1330633v1_protein 3824796
GCF_013306435.1_ASM1330643v1_protein 3826515
GCF_013306935.1_ASM1330693v1_protein 3824484
GCF_013375735.1_ASM1337573v1_protein 3827109


# Convert to Phylip

In [65]:
SeqIO.convert(msa_out_fasta, "fasta", msa_out_phylip, "phylip")

6

# Export XMFA

In [66]:
msa_out_xmfa = open(msa_out_xmfa, 'w')

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    for record in SeqIO.parse(pmsa, "fasta"):
        #print(record.id)
        g_id = record.id.split('_WP')[0].replace('_1_', '.1_')
        p_id = record.id.split('_protein_')[-1]
        #print(g_id, p_id)
        nns = prot_db.loc[p_id, g_id]['nucl']
        #print('fffffff',  nns)
        msa_out_xmfa.write('>'+record.id.split('_')[3][:10]+'\n')
        
        atcg = {'A', 'T', 'G', 'C','-'}
        ambig = set(nns).difference(atcg)
        #print(ambig)
        for b in ambig:
            nns = str(nns).replace(b, '-')
        #print('aaaaaa',  nns)
        
        gaps_v = gaps2tab.loc[og, g_id]
        i = 0
        for gap in gaps_v:
            if gap:
                msa_out_xmfa.write('-')
            else:
                msa_out_xmfa.write(nns[i])
                i = i + 1
        msa_out_xmfa.write('\n')
    msa_out_xmfa.write('=' + '\n')
msa_out_xmfa.close()

# Test the position mapping
+ grab sequences from the original gb files using g2pos, g2strand and g2chr
+ load exported core OG nucleotide fasta file
+ compare the sequences 

(note: reverse compliment!)

In [67]:
if DEBUG:
    # for the compliment strand mapping
    from Bio.Seq import Seq
    Seq("A").complement()

In [68]:
if DEBUG:
    import os
    gbks = '/Users/devseeva/Desktop/work/pan-genome-visualization/data/rso_test/input_GenBank'
    ccc=0

    test_fasta = {}
    for entry in os.scandir(gbks):
        #if ccc>0:
        #    break
        ccc = ccc+1
        if (entry.path.endswith(".gbk") and entry.is_file()):
            genome = os.path.basename(entry).replace('_genomic.gbk', '_protein')
            test_fasta[genome] = []
            print('Genome:', genome)

            # map chr id to the sequence
            chr_seq = {}
            for record in SeqIO.parse(entry.path, "gb"):
                print('Chr',record.id, 'with', len(record.features), 'features')
                chr_seq[record.id] = record.seq

            # get corresponding mapping (from above)
            mapping_chr = g2chr[genome]
            mapping_pos = g2pos[genome]
            mapping_str = g2strand[genome]
            assert(len(mapping_chr) == len(mapping_pos))

            # iterate through the original positions 
            for i in range(len(mapping_chr)):
                if mapping_str[i] == 1:
                    test_fasta[genome].append(chr_seq[mapping_chr[i]][mapping_pos[i]])
                else:
                    compl = str(Seq(chr_seq[mapping_chr[i]][mapping_pos[i]]).complement())
                    test_fasta[genome].append(compl)
            test_fasta[genome] = "".join(test_fasta[genome])
            print("Core nucl exported:", len(test_fasta[genome]))

            print()
            

In [69]:
if DEBUG:

    for record in SeqIO.parse("core_OG_nucl.fasta", "fasta"):
        print(record.id, len(record.seq))
        print('test length', len(record.seq))
        print('Test:', test_fasta[record.id] == str(record.seq))
        print()


f = "/Users/devseeva/Desktop/work/pan-genome-visualization/data/ncbi-rso/metainfo.tsv"
x = pd.read_csv(f, sep='\t')  
phys = []
for a in x["accession"]:
    phys.append(a.split(".1_")[1][:10])
x["phy_id"] = phys
x.to_csv("/Users/devseeva/Desktop/work/sm_workflow/snakefiles/inputs/phy4_test_metadata/phy4_test_cleaned_data.csv", index=False)